In [1]:
import numpy as np

In [2]:
POP_SIZE = 20
CHRO_LENGTH = 10
MU_RATE = 0.1
GENE_NUM = 20

In [3]:
class Chromosome:
    def __init__(self):
        self.genes = np.random.randint(2, size=CHRO_LENGTH)
        
    def Fitness(self):
        return np.sum(self.genes)
    
    def mutate(self, p):
        if np.random.random() < p:
            point = np.random.randint(0, CHRO_LENGTH)
            self.genes[point] += 1
            self.genes[point] %= 2

In [4]:
class Population:
    def __init__(self):
        self.chromosomes = []
        for i in range(POP_SIZE):
            self.chromosomes.append(Chromosome())
        self.fitness_update()
        self.generation = 1
    
    def __repr__(self):
        return f'Gen : {self.generation}, F : {self.f}, Avg F : {self.f.mean()}'
    
    def fitness_update(self):
        self.f = [c.Fitness() for c in self.chromosomes]
        self.f = np.array(self.f)
    
    def roulette_update(self, k):
        cb, cw = self.f.max(), self.f.min()
        if cb-cw == 0:
            self.r_fi = np.full(POP_SIZE, 1/POP_SIZE)
            return
        self.r_fi = [(ci-cw) + (cb-cw) / (k-1) for ci in self.f]
        self.r_fi = np.array(self.r_fi)
        self.r_fi /= self.r_fi.sum()

    def roulette_selection(self, n):
        return np.random.choice(POP_SIZE, n, p=self.r_fi)
    
    def crossover(self, i1, i2):
        point = np.random.randint(0, CHRO_LENGTH)
        offspring = Chromosome()
        g1 = self.chromosomes[i1].genes[:point]
        g2 = self.chromosomes[i2].genes[point:]
        offspring.genes = np.append(g1, g2)
        offspring.mutate(MU_RATE)
        return offspring
        
    # strady-state GA
    def next_generation(self):
        self.roulette_update(4)
        selected = self.roulette_selection(POP_SIZE)
        offsprings = []
        for s1, s2 in zip(selected[::2], selected[1::2]):
            offsprings.append(self.crossover(s1, s2))
        replaced = np.argsort(self.f)
        for i in range(int(POP_SIZE/2)):
            self.chromosomes[replaced[i]] = offsprings[i]
        self.generation += 1
        self.fitness_update()

In [5]:
p = Population()
print(p)

for i in range(GENE_NUM-1):
    p.next_generation()
    print(p)

Gen : 1, F : [6 4 4 4 6 4 5 2 8 5 4 7 6 3 6 5 6 4 7 6], Avg F : 5.1
Gen : 2, F : [6 9 6 7 6 6 7 5 8 4 8 7 6 3 6 5 6 5 7 6], Avg F : 6.15
Gen : 3, F : [8 9 6 7 6 8 7 6 8 6 8 7 5 6 5 7 5 5 7 6], Avg F : 6.6
Gen : 4, F : [8 9 5 7 6 8 7 9 8 7 8 7 5 8 4 7 7 7 7 7], Avg F : 7.05
Gen : 5, F : [8 9 7 7 5 8 7 9 8 4 8 8 8 8 7 8 8 9 7 7], Avg F : 7.5
Gen : 6, F : [8 9 7 8 8 8 9 9 9 7 7 8 8 8 9 8 8 9 8 7], Avg F : 8.1
Gen : 7, F : [8 9 8 8 8 8 9 9 9 9 8 8 8 9 9 8 8 9 8 8], Avg F : 8.4
Gen : 8, F : [8 9 8 8 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9], Avg F : 8.85
Gen : 9, F : [9 9 9 8 9 9 9 9 9 9 9 9 8 9 9 9 9 9 9 9], Avg F : 8.9
Gen : 10, F : [ 9  9  9 10  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9], Avg F : 9.05
Gen : 11, F : [10  9  9 10  9  9  9  9  9  9 10  9  9  9  9  9 10  9  9  9], Avg F : 9.2
Gen : 12, F : [10  9  9 10  9  9  9  9  9 10 10  9  9  9  9  8 10  9  9 10], Avg F : 9.25
Gen : 13, F : [10 10  9 10  9  9 10 10  8 10 10  9  9  9  9  9 10 10  9 10], Avg F : 9.45
Gen : 14, F : [10 10  8 10